## Digit Recognizer

### Importing Essential Packages 📦🔍

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import random
from tensorflow.keras import layers, models

### Importing data 📦📦

In [ ]:
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')
submission = pd.read_csv('../data/sample_submission.csv')

In [ ]:
train_df.shape, test_df.shape, submission.shape

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
submission.head()

In [ ]:
train_df.info()

In [ ]:
test_df.info()

In [ ]:
submission.info()

### Creating Images from Numbers 📸🔢

In [ ]:
number = random.randint(0, 42000) 
number

In [ ]:
plt.figure(figsize=(1, 1))
plt.imshow(train_df.iloc[number, 1:].values.reshape(28, 28), cmap=plt.cm.binary)

In [ ]:
# code to show 100 images
plt.figure(figsize=(8, 8))
for i in range(100):
    plt.subplot(10, 10, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(train_df.iloc[i, 1:].values.reshape(28, 28), cmap=plt.cm.binary)
    plt.xlabel(train_df.iloc[i, 0])

In [ ]:
# converting number 0 to 256 to 0 to 1
x_train_df = train_df.iloc[:, 1:].values / 256
x_test_df = test_df.iloc[:, :].values / 256
x_train_df[number]

In [ ]:
y_train_df = train_df.iloc[:, 0].values
y_train_df[number]

In [ ]:
x_train_df.shape, x_test_df.shape

In [ ]:
x_train_df[0]

### Building and Training Models 🏗️🤖 

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_train_df, y_train_df, test_size=0.2, random_state=42)

In [ ]:
x_train[0]

In [ ]:
y_train[0]

#### Artificial Neural Network (ANN) 🧠🔗

In [ ]:
model=keras.Sequential([
    keras.layers.Dense(128, input_shape=(784,), activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(10, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, epochs=10,
                    batch_size=32, validation_data=(x_test, y_test))

In [ ]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.head()

In [ ]:
plt.figure(figsize=(4, 4))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')

plt.show()

In [ ]:
y_predicted = model.predict(x_test)
y_predicted[0]

In [ ]:
plt.figure(figsize=(1, 1))
plt.imshow(x_test[0].reshape(28, 28), cmap=plt.cm.binary)

In [ ]:
np.argmax(y_predicted[0])

In [ ]:
y_predicted_labels = [np.argmax(i) for i in y_predicted]

In [ ]:
y_predicted_labels[:20]

In [ ]:
cm = tf.math.confusion_matrix(labels=y_test,predictions=y_predicted_labels)
cm

In [ ]:
import seaborn as sn
plt.figure(figsize = (7,5))
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

#### Convolutional Neural Network (CNN) 🌐🔍

In [ ]:
cnn_model = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3),
                  activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(10, activation='softmax')
])

In [ ]:
cnn_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = cnn_model.fit(x_train.reshape(-1, 28, 28, 1), y_train, epochs=10,
              batch_size=32, validation_data=(x_test.reshape(-1, 28, 28, 1), y_test))

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.head()

In [ ]:
plt.figure(figsize=(4, 4))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')

plt.show()

In [ ]:
test_loss, test_accuracy = cnn_model.evaluate(x_test.reshape(-1, 28, 28, 1), y_test)
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
y_predicted = cnn_model.predict(x_test.reshape(-1, 28, 28, 1))

In [ ]:
y_clsses = [np.argmax(i) for i in y_predicted]
y_clsses[:20]

In [ ]:
y_test[:20]

In [ ]:
plt.figure(figsize=(8, 8))
count = 0  # Initialize a count variable

for i in range(x_test.shape[0]):
    if y_clsses[i] != y_test[i]:
        count += 1
        if count > 100:
            break  # Exit the loop if more than 100 subplots are created
        plt.subplot(10, 10, count)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(x_test[i].reshape(28, 28), cmap=plt.cm.binary)
        plt.xlabel(y_clsses[i])
        plt.ylabel(y_test[i])

plt.show()

In [ ]:
cm = tf.math.confusion_matrix(labels=y_test,predictions=y_clsses)
cm

In [ ]:
import seaborn as sn
plt.figure(figsize = (7,5))
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

#### Making Predictions 🚀🔮

In [ ]:
y_predicted_x_test_df = cnn_model.predict(x_test_df.reshape(-1, 28, 28, 1))
y_predicted_x_test_df

In [ ]:
y_classes_x_test_df = [np.argmax(i) for i in y_predicted_x_test_df]
y_classes_x_test_df[:20]

In [ ]:
result_df = pd.DataFrame({
    "ImageId": submission["ImageId"],
    "Label": y_classes_x_test_df
})

result_df.to_csv("../data/predictions.csv", index=False)

print(result_df.head())

#### Implementing Dropout for Regularization 🕳️🔄

In [ ]:
cnn_model_dropout = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3),
                  activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(16, activation='relu'),
    layers.Dense(10, activation='softmax')
])

In [ ]:
cnn_model_dropout.compile(optimizer='adam',
                    loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = cnn_model_dropout.fit(x_train.reshape(-1, 28, 28, 1), y_train, epochs=10,
                batch_size=32, validation_data=(x_test.reshape(-1, 28, 28, 1), y_test))

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.head()

In [ ]:
test_loss, test_accuracy = cnn_model_dropout.evaluate(x_test.reshape(-1, 28, 28, 1), y_test)

In [ ]:
y_predicted = cnn_model_dropout.predict(x_test.reshape(-1, 28, 28, 1))

In [ ]:
y_clsses = [np.argmax(i) for i in y_predicted]
y_clsses[:20]

In [ ]:
y_test[:20]

In [ ]:
plt.figure(figsize=(8, 8))
count = 0  # Initialize a count variable

for i in range(x_test.shape[0]):
    if y_clsses[i] != y_test[i]:
        count += 1
        if count > 100:
            print(count)
            break  # Exit the loop if more than 100 subplots are created
        plt.subplot(10, 10, count)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(x_test[i].reshape(28, 28), cmap=plt.cm.binary)
        plt.xlabel(y_clsses[i])
        plt.ylabel(y_test[i])

plt.show()

#### Data Augmentation for Improved Generalization 🔄📸

In [ ]:
data_augmentation = keras.Sequential([
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
    layers.experimental.preprocessing.RandomContrast(0.1)
])

In [ ]:
model_data_augmentation = models.Sequential([
    data_augmentation,
    layers.Conv2D(filters=32, kernel_size=(3, 3),
                  activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(10, activation='softmax')
])

In [ ]:
model_data_augmentation.compile(optimizer='adam',
                                loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history =  model_data_augmentation.fit(x_train.reshape(-1, 28, 28, 1), y_train, epochs=10,
                            batch_size=32, validation_data=(x_test.reshape(-1, 28, 28, 1), y_test))

In [ ]:
test_loss, test_accuracy = model_data_augmentation.evaluate(x_test.reshape(-1, 28, 28, 1), y_test)
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
y_predicted = model_data_augmentation.predict(x_test.reshape(-1, 28, 28, 1))

In [ ]:
y_classes = [np.argmax(i) for i in y_predicted]
y_clsses[:20]

In [ ]:
y_test[:20]

In [ ]:
cm = tf.math.confusion_matrix(labels=y_test,predictions=y_clsses)
cm

In [ ]:
import seaborn as sn
plt.figure(figsize = (7,5))
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
plt.figure(figsize=(8, 8))
count = 0  # Initialize a count variable

for i in range(x_test.shape[0]):
    if y_classes[i] != y_test[i]:
        count += 1
        if count > 100:
            print(count)
            break  # Exit the loop if more than 100 subplots are created
        plt.subplot(10, 10, count)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(x_test[i].reshape(28, 28), cmap=plt.cm.binary)
        plt.xlabel(y_classes[i])
        plt.ylabel(y_test[i])

plt.show()

### Conclusion 🎉🏁

#### **Score: 0.9876**